In [1]:
#imports
import glob
import pickle
from tqdm.notebook import tqdm
import os

In [2]:
#PATHs

USER = "leila" #change the user 
DATA_ROOT = "/autofs/space/simba_001/users/PBR28_KOALA/data/PET"
WORKING_PROCESSED_DATA_PATH = f"/autofs/space/celer_001/users/{USER}/working_pbr28"
WINDOWS_SHARED_DATA_PATH =  f"/autofs/space/celer_001/users/{USER}/pbr28"
PICKLE_SAVE_PATH = f"{WORKING_PROCESSED_DATA_PATH}/pickles"

In [3]:
#PARAMS

NUMBER_OF_SAMPLES = 30

In [5]:
all_pets = glob.glob(f"{DATA_ROOT}/**/")

In [6]:
# Load patients with GTs 
with open(f"{PICKLE_SAVE_PATH}/unfiltered_patient_list.pkl" , "rb") as f:
    gt_patients = pickle.load(f)

In [7]:
# Copy the filtered paths

filtered_pt_paths = [f for f in all_pets if f.split("/")[8] in gt_patients]
final_filtered_pats = []

def insensitive_glob(pattern):
    def either(c):
        return '[%s%s]' % (c.lower(), c.upper()) if c.isalpha() else c
    return glob.glob(''.join(map(either, pattern)))

for f in tqdm(filtered_pt_paths):
    list_mode_paths = insensitive_glob(f"{f}/ListMode*/")
    attenuation_paths = insensitive_glob(f"{f}/*muMAP*/")
    if len(list_mode_paths) != 0 and len(attenuation_paths) != 0:
        final_filtered_pats.append(f)

  0%|          | 0/112 [00:00<?, ?it/s]

In [ ]:
for i, f in enumerate(tqdm(final_filtered_pats)):
    pat_id = f.split("/")[-2]
    pat_path = f"{WINDOWS_SHARED_DATA_PATH}/{pat_id}"

    list_mode_path = insensitive_glob(f"{f}/ListMode*/")[0]
    attenuation_path = insensitive_glob(f"{f}/*muMAP*/")[0]
    os.makedirs(pat_path, exist_ok=True)
    
    list_mode_name = list_mode_path.split("/")[-2].replace("/" , "")
    attenuation_name = attenuation_path.split("/")[-2].replace("/" , "")
    
    !cp -rf "$list_mode_path/30001Head_PetAcquisition_Raw_Data" "$pat_path"
    !cp -rf "$attenuation_path" "$pat_path/30001Head_PetAcquisition_Raw_Data/"
    !mv "$pat_path/30001Head_PetAcquisition_Raw_Data/$attenuation_name" "$pat_path/30001Head_PetAcquisition_Raw_Data/CT"
    
    if i == NUMBER_OF_SAMPLES:
        break

  0%|          | 0/107 [00:00<?, ?it/s]

cp: cannot stat '/autofs/space/simba_001/users/PBR28_KOALA/data/PET/PBRKOA_KOA001_091317/Listmode_Data//30001Head_PetAcquisition_Raw_Data': No such file or directory
mv: cannot stat '/autofs/space/celer_001/users/leila/pbr28/PBRKOA_KOA001_091317/30001Head_PetAcquisition_Raw_Data/pseudo_muMAP': No such file or directory
cp: cannot stat '/autofs/space/simba_001/users/PBR28_KOALA/data/PET/PBRKOA_KOA035_011020/Listmode//30001Head_PetAcquisition_Raw_Data': No such file or directory
mv: cannot stat '/autofs/space/celer_001/users/leila/pbr28/PBRKOA_KOA035_011020/30001Head_PetAcquisition_Raw_Data/pseudo_muMAP': No such file or directory


In [39]:
!ls "$WINDOWS_SHARED_DATA_PATH"

30001Head_PetAcquisition_Raw_Data  30001Head_PetAcquisition_Raw_Data-Converted
